In [1]:
import csv
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
import tabula
import numpy as np
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter

In [2]:
# Pasta contendo os arquivos PDF
pasta_eventos = "Eventos"

# Nomes das colunas
nomes_colunas = [
    "Código do Relatório",
    "Concelho origem do incidente",
    "Data do incidente",
    "Nível de Tensão",
    "Causa do incidente",
    "Duração incidente (min)",
    "N.º clientes afetados",
    "TIEPI MT (min)",
    "SAIFI MT (#)",
    "SAIDI MT (min)",
    "END (MWh)",
    "SAIFI BT (#)",
    "SAIDI BT (min)",
    "Qualidade de Energia Elétrica",
    "Qualidade de Serviço Comercial",
    "Fundamentação", 
    "Decisão",
]

# DataFrame para armazenar os dados
df_eventos = pd.DataFrame()

# Iterar sobre os arquivos na pasta "Eventos"
for arquivo in os.listdir(pasta_eventos):
    if arquivo.endswith(".pdf"):
        caminho_arquivo = os.path.join(pasta_eventos, arquivo)
        lista_tabelas = tabula.read_pdf(caminho_arquivo, pages='all', lattice=True)
        for tabela in lista_tabelas:
            tabela = tabela.iloc[3:]
            # if "Relatório" not in tabela.columns[0]:
            #     pass
            if len(tabela.columns) > 10 and tabela.shape[0] > 0:
                try:
                    # Tenta converter a última coluna para datetime
                    tabela.iloc[:, -1] = pd.to_datetime(tabela.iloc[:, -1])
                    # Remove a última coluna
                    tabela = tabela.iloc[:, :-1]
                except ValueError:
                    pass
                    
                if len(tabela.columns) == 16:
                    tabela.insert(13, 'Wow', np.nan)


                if len(tabela.columns) == 18:
                    try:
                        if tabela.columns[3] == "Código do\rIncidente":
                            tabela = tabela.drop(tabela.columns[3], axis=1)
                        else:
                            tabela.iloc[:, 3] = pd.to_numeric(tabela.iloc[:, 3])
                            tabela = tabela.drop(tabela.columns[3], axis=1)
                    except ValueError:
                        pass

                if len(tabela.columns) == 17:
                    try:
                        if tabela.columns[3] == "Código do\rIncidente":
                            tabela = tabela.drop(tabela.columns[3], axis=1)
                            tabela.insert(13, 'Wow', np.nan)
                        else:
                            tabela.iloc[:, 3] = pd.to_numeric(tabela.iloc[:, 3])
                            tabela = tabela.drop(tabela.columns[3], axis=1)
                            tabela.insert(13, 'Wow', np.nan)
                    except ValueError:
                        pass  # Ignorar a exceção e continuar

                # Nomear as colunas
                tabela.columns = nomes_colunas
                # Adicionar a tabela ao DataFrame
                df_eventos = pd.concat([df_eventos, tabela], ignore_index=True)

# Exibir o DataFrame
df_eventos


C:\Users\diogo\AppData\Local\Temp\ipykernel_9160\974119757.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tabela.iloc[:, -1] = pd.to_datetime(tabela.iloc[:, -1])
C:\Users\diogo\AppData\Local\Temp\ipykernel_9160\974119757.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tabela.iloc[:, -1] = pd.to_datetime(tabela.iloc[:, -1])
C:\Users\diogo\AppData\Local\Temp\ipykernel_9160\974119757.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tabela.iloc[:, -1] = pd.to_datetime(tabela.iloc[:, -1])
C:\Users\diogo\AppData\Local\Temp\ipykernel_9160\974119757.py:40: UserWarnin

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão
0,EDPD_2018_JAN_T_1,Almada,02/01/2018,MT,ESCAVAÇÕES,55.0,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação:\r- Após pa...,Não Aprovado
1,EDPD_2018_JAN_T_2,Serta,02/01/2018,MT,ABATE DE ÁRVORES,163.0,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação:\r- Reúne os re...,Aprovado
2,EDPD_2018_JAN_T_3,Felgueiras,04/01/2018,MT,DESLIZAMENTO DE TERRAS,596.0,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação:\r- Reúne os re...,Aprovado
3,EDPD_2018_JAN_T_4,Torres Vedras,05/01/2018,BT,VEÍCULOS,178.0,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação:\r- Após pa...,Não Aprovado
4,EDPD_2018_JAN_T_5,Nisa,09/01/2018,MT,AVES,119.0,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação:\r- Reúne os re...,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7974,EDPD_2017_NOV_T_62,Ovar,29/11/2017,BT,VEÍCULOS,220,2,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação:\r- Não cum...,Não Aprovado
7975,EDPD_2017_NOV_T_63,Santa Maria da Feira,03/11/2017,BT,VEÍCULOS,92,1,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação:\r- Não cum...,Não Aprovado
7976,EDPD_2017_NOV_T_64,Santa Maria da Feira,23/11/2017,BT,VEÍCULOS,121,1,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação:\r- Não cum...,Não Aprovado
7977,EDPD_2017_NOV_T_65,Almeirim,03/11/2017,BT,DESCARGA ATMOSFÉRICA DIRETA,154,1,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação:\r- Não cum...,Não Aprovado


In [3]:
df_eventos = df_eventos.applymap(lambda x: x.replace('\r', ' ') if isinstance(x, str) else x)
df_eventos["Concelho origem do incidente"] = df_eventos["Concelho origem do incidente"].str.title()
df_eventos["Causa do incidente"] = df_eventos["Causa do incidente"].str.title()
df_eventos

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão
0,EDPD_2018_JAN_T_1,Almada,02/01/2018,MT,Escavações,55.0,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado
1,EDPD_2018_JAN_T_2,Serta,02/01/2018,MT,Abate De Árvores,163.0,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
2,EDPD_2018_JAN_T_3,Felgueiras,04/01/2018,MT,Deslizamento De Terras,596.0,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
3,EDPD_2018_JAN_T_4,Torres Vedras,05/01/2018,BT,Veículos,178.0,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado
4,EDPD_2018_JAN_T_5,Nisa,09/01/2018,MT,Aves,119.0,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7974,EDPD_2017_NOV_T_62,Ovar,29/11/2017,BT,Veículos,220,2,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7975,EDPD_2017_NOV_T_63,Santa Maria Da Feira,03/11/2017,BT,Veículos,92,1,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7976,EDPD_2017_NOV_T_64,Santa Maria Da Feira,23/11/2017,BT,Veículos,121,1,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7977,EDPD_2017_NOV_T_65,Almeirim,03/11/2017,BT,Descarga Atmosférica Direta,154,1,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado


In [4]:
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('MT ‐ Média tensão', 'MT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('MT - Média tensão', 'MT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('MT ‐ Média\rtensão', 'MT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('BT ‐ Baixa tensão', 'BT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('BT - Baixa tensão', 'BT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('AT ‐ Alta tensão', 'AT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('AT; MT; BT', 'AT, MT, BT', regex=True)
df_eventos['Nível de Tensão'] = df_eventos['Nível de Tensão'].replace('MT/BT', 'MT, BT', regex=True)

for i in df_eventos["Nível de Tensão"].unique():
    if "Tipo" in i:
        linha = df_eventos[df_eventos["Nível de Tensão"] == i].index
        df_eventos.loc[linha, "Nível de Tensão"] = "BT"

df_eventos["Nível de Tensão"].unique()

array(['MT', 'BT', 'AT', 'AT - Alta tensão', 'eredes', 'MT, BT', 'I',
       'AT, MT, BT'], dtype=object)

In [5]:
for i in df_eventos["Causa do incidente"].unique():
    if "rvores" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Abate De Árvores"

    elif "Neve" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Neve/Gelo/Granizo Condição Excecional"

    elif "Malfeitoria" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Malfeitoria/Vandalismo"

    elif "Risco" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Risco Iminente Pessoas Ou Bens"

    elif "Descarga" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Descarga Atmosférica Directa"

    elif "Escavações" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Escavações"

    elif "Vento Intensidade" in i:
        linha = df_eventos[df_eventos["Causa do incidente"] == i].index
        df_eventos.loc[linha, "Causa do incidente"] = "Vento Intensidade Excepcional"

df_eventos["Causa do incidente"].unique()

array(['Escavações', 'Abate De Árvores', 'Deslizamento De Terras',
       'Veículos', 'Aves', 'Animais Não Aves', 'Incêndio',
       'Malfeitoria/Vandalismo', 'Terramoto',
       'Neve/Gelo/Granizo Condição Excecional',
       'Corpos Estranhos Na Rede', 'Vento Intensidade Excepcional',
       'Descarga Atmosférica Directa', 'Intervenções À Superficie',
       'Inundações Imprevisíveis', 'Furtos',
       'Risco Iminente Pessoas Ou Bens', 'Sabotagem',
       'Desabamento De Muros/Outras Cons',
       'Deslastre De Carga Automático',
       'Desabamento De Muros/Outras Construções',
       'Vento E Chuva Intensidade Excecional',
       'Trabalhos De Terceiros ( Gruas,... )',
       'Ruptura De Canal. De Fluídos', 'Malfeitorioa (Vandalismo)',
       'Rnt - Caso Fortuito/Força Maior/Razão Segurança'], dtype=object)

In [6]:
df_eventos.to_excel('EventosExcecionaisNew.xlsx', index=False, header=True)